In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )

        return data_transformation_config

In [6]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [31]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def pipeline(self):
        data = pd.read_csv(self.config.data_path)
        cat_features = data.select_dtypes(include="object").columns
        encoder = OneHotEncoder()

        encoded_features = encoder.fit_transform(data[cat_features])

        data = pd.get_dummies(data, columns=cat_features)
        self.data = data
        print(data.head(1))
        logger.info("Pipeline applied")
        logger.info(data.shape)
        print(data.shape)

    def train_test_spliting(self):
        data = self.data
        # Split the data into training and test sets. (0.80, 0.20) split.
        train, test = train_test_split(data,test_size=0.2,random_state=42)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)
        

In [32]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.pipeline()
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2023-08-08 18:02:26,991: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-08 18:02:26,992: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-08 18:02:26,994: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-08 18:02:26,995: INFO: common: created directory at: artifacts]
[2023-08-08 18:02:26,996: INFO: common: created directory at: artifacts/data_transformation]
    Price  Year  Kilometer  Engine  Length   Width  Height  Seating Capacity  \
0  505000  2017      87150  1198.0  3990.0  1680.0  1505.0               5.0   

   Fuel Tank Capacity  TorquePower  ...  Owner_Fourth  Owner_Second  \
0                35.0        109.0  ...         False         False   

   Owner_Third  Owner_UnRegistered Car  Seller Type_Commercial Registration  \
0        False                   False                                False   

   Seller Type_Corporate  Seller Type_Individual  Drivetrain_AWD  \
0                   True                   F